Задание по теме регрессии

- Формулировка задания:<br>
Решить задачу регрессии. <br>
Получить максимальную метрику качества (R2, RMSE).<br>

Введение<br>
Целью данной задачи является прогнозирование суммы пятничных распродаж в магазинах сети Walmart с помощью построения регрессионных моделей и их анализа.<br>
Набор данных состоит из исторических данных, которые охватывают продажи с 2010-02-05 по 2012-11-01, предствленных в файле Walmart.csv<br>
#Использовать приплайны. <br>
#Выполнить поиск лучших параметров / *hyperopt / **optuna. <br>
#Кроссвалидация / различные типы проверок. <br>
#Сохранить и загрузить моделль. <br>
#H20 (если получиться) <br>
<br>
**
<br>
- Анализ и план выполнения задания.<br>
Отметим, что в файле стоят даты - сплошь пятницы. В задании требуется найти суммы именно пятничных продаж. При этом почему-то графа с продажами имеет название Weekly_Sales. Можно, конечно, считать, что это продажи за всю неделю, начиная (или заканчивая) упомянутыми пятницами. Но как тогда предсказывать суммы пятничных распродаж? Поэтому переименуем колонку в Sales и будем смело полагать, что это суммы продаж именно за пятницу.<br>

Следует загрузить файл csv датасета, используя таблицу pandas.<br>
Произвести предобработку данных. <br>
Выделить независимые переменные - характеристики (features) модели.<br>
Выделить колонку целевой (target) переменной - суммы пятничных продаж.<br>
Разделить данные на тренировочный и тестовый датасет.<br>
Создать модель с использованием пайплайна.<br>
Подобрать гиперпараметры.<br>
Сохранить модель в файл.<br>
Выгрузить обратно из файла.<br>
Провести проверку модели на тестовой выборке. Рассчитать метрики на тестовой выборке. <br>

Загрузка необходимых библиотек

In [26]:
# %reset
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error, r2_score
import joblib
import optuna
from optuna import create_study
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

Загрузка данных

In [27]:
# Load data
data = pd.read_csv('Walmart.csv')

# Viewing first strings:
print(data.head())

   Store        Date       Sales  Holiday_Flag  Temperature  Fuel_Price  \
0      1  05-02-2010  1643690.90             0        42.31       2.572   
1      1  12-02-2010  1641957.44             1        38.51       2.548   
2      1  19-02-2010  1611968.17             0        39.93       2.514   
3      1  26-02-2010  1409727.59             0        46.63       2.561   
4      1  05-03-2010  1554806.68             0        46.50       2.625   

          CPI  Unemployment  
0  211.096358         8.106  
1  211.242170         8.106  
2  211.289143         8.106  
3  211.319643         8.106  
4  211.350143         8.106  


Среди данных есть колонка с датами. Для построения модели регрессии дата не нужна.
Остальные параметры - номер магазина, совпадение с праздником, температура, стоимость топлива, индекс потребительских цен, уровень безработицы, - являются потенциально важными для модели.

In [28]:
# check colums types
print(f'Data types: {data.dtypes}')

Data types: Store             int64
Date             object
Sales           float64
Holiday_Flag      int64
Temperature     float64
Fuel_Price      float64
CPI             float64
Unemployment    float64
dtype: object


Проверяем, требуется ли обработка пропусков

In [29]:
print('How many omitted data in each column:')
print(data.isna().sum())

How many omitted data in each column:
Store           0
Date            0
Sales           0
Holiday_Flag    0
Temperature     0
Fuel_Price      0
CPI             0
Unemployment    0
dtype: int64


Пропущенных данных нет.<br>
Проводим разделение выборки на тренировочную и тестовую подвыборки. Попутно выбрасываем колонку с датами.

In [30]:
# Assume 'Sales' is the target variable and others are features
X = data.drop('Sales', axis=1)
y = data['Sales']

# Removing dates from the model:
X = data.drop('Date', axis=1)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Поиск оптимальных значений гиперпараметров с помощью optuna

In [31]:
# Define a function to hyperparameters optimization by optuna
def objective_optuna(trial):
    # Hyperparameter tuning for RandomForestRegressor
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 5, 50)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)

    # Create a pipeline
    pipe = Pipeline([
        ('scaler', StandardScaler()),  # Feature scaling
        ('model', RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth,
                                         min_samples_split=min_samples_split, random_state=42))
    ])
    
    # Perform cross-validation
    scores = cross_val_score(pipe, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    rmse = root_mean_squared_error(y_train, pipe.fit(X_train, y_train).predict(X_train))
    
    return rmse

In [32]:
# Create a study for hyperparameter optimization with optuna
n_max = 50
study = create_study(direction='minimize')
study.optimize(objective_optuna, n_trials = n_max)

[I 2025-02-05 21:15:54,589] A new study created in memory with name: no-name-0acaaad0-f72b-4664-8a8e-dc83547da8ba
[I 2025-02-05 21:15:58,023] Trial 0 finished with value: 6371.260599248632 and parameters: {'n_estimators': 57, 'max_depth': 37, 'min_samples_split': 10}. Best is trial 0 with value: 6371.260599248632.
[I 2025-02-05 21:16:04,540] Trial 1 finished with value: 6007.334394936256 and parameters: {'n_estimators': 107, 'max_depth': 13, 'min_samples_split': 9}. Best is trial 1 with value: 6007.334394936256.
[I 2025-02-05 21:16:12,057] Trial 2 finished with value: 5296.624922103219 and parameters: {'n_estimators': 121, 'max_depth': 35, 'min_samples_split': 8}. Best is trial 2 with value: 5296.624922103219.
[I 2025-02-05 21:16:20,674] Trial 3 finished with value: 4292.603441216142 and parameters: {'n_estimators': 136, 'max_depth': 15, 'min_samples_split': 7}. Best is trial 3 with value: 4292.603441216142.
[I 2025-02-05 21:16:26,358] Trial 4 finished with value: 3464.247814128804 and

Создаем модель с наилучшими гиперпараметрами по версии optuna

In [33]:
# the best model according to optuna
best_params_optuna = study.best_params
best_model_optuna = Pipeline([
    ('scaler', StandardScaler()),
    ('model', RandomForestRegressor(**best_params_optuna, random_state=42))
])
best_model_optuna.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('model',
                 RandomForestRegressor(max_depth=20, n_estimators=144,
                                       random_state=42))])

Оценка модели на тестовой выборке

In [34]:
# Evaluate the model
y_pred = best_model_optuna.predict(X_test)
print(f"Optuna's  R^2 Score: {r2_score(y_test, y_pred)}")
print(f"Optuna's RMSE: {root_mean_squared_error(y_test, y_pred)}")

Optuna's  R^2 Score: 0.9997868786591816
Optuna's RMSE: 8286.01308021603


Демонстрация сохранения модели в файлик с целью последующей загрузки из файла

In [35]:

# Save the model to a file
joblib.dump(best_model_optuna, 'best_walmart_model_optuna.pkl')


['best_walmart_model_optuna.pkl']

Поиск оптимальных гиперпараметров с помощью hyperopt

Функция для поиска наилучшего набора гиперпараметров с помощью hyperopt

In [36]:
# Hyperopt hyperparameter optimization
def objective_hyperopt(params):
    n_estimators = int(params['n_estimators'])
    max_depth = int(params['max_depth'])
    min_samples_split = int(params['min_samples_split'])

    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('model', RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth,
                                         min_samples_split=min_samples_split, random_state=42))
    ])

    rmse = -cross_val_score(pipe, X_train, y_train, cv=5, scoring='neg_mean_squared_error').mean()
    return {'loss': rmse, 'status': STATUS_OK}

In [37]:
# Define the search space for Hyperopt
space = {
    'n_estimators': hp.quniform('n_estimators', 50, 150, 1),          # Range 50 to 200
    'max_depth': hp.quniform('max_depth', 5, 50, 1),                  # Range 5 to 50
    'min_samples_split': hp.quniform('min_samples_split', 2, 10, 1)   # Range 2 to 10
}

Поиск гиперпараметров с помощью hyperopt

In [38]:
# Run Hyperopt optimization
trials = Trials()
best_hyperopt = fmin(fn=objective_hyperopt, space=space, algo=tpe.suggest, max_evals = n_max, trials=trials)

100%|██████████| 50/50 [04:42<00:00,  5.64s/trial, best loss: 23585159.576082714]


In [39]:
# Take the best model for Hyperopt
best_params_hyperopt = {
    'n_estimators': int(best_hyperopt['n_estimators']),
    'max_depth': int(best_hyperopt['max_depth']),
    'min_samples_split': int(best_hyperopt['min_samples_split'])
}

In [40]:
best_params_hyperopt

{'n_estimators': 150, 'max_depth': 10, 'min_samples_split': 3}

Передача найденного наилучшего набора параметров в модель.
Поиск параметров модели регрессии, т.е. тренировка модели на тренировочной выборке.

In [41]:
best_model_hyperopt = Pipeline([
    ('scaler', StandardScaler()),
    ('model', RandomForestRegressor(**best_params_hyperopt, random_state=42))
])
best_model_hyperopt.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('model',
                 RandomForestRegressor(max_depth=10, min_samples_split=3,
                                       n_estimators=150, random_state=42))])

Оценка модели (с набором гиперпараметров от hyperopt) на тестовой выборке

In [42]:
# Evaluate the Hyperopt model
y_pred_hyperopt = best_model_hyperopt.predict(X_test)
print(f'Hyperopt Model R^2 Score: {r2_score(y_test, y_pred_hyperopt)}')
print(f'Hyperopt Model RMSE: {root_mean_squared_error(y_test, y_pred_hyperopt)}')

Hyperopt Model R^2 Score: 0.9997638020230121
Hyperopt Model RMSE: 8723.087552232219


In [43]:
# Save the Hyperopt model to a file
joblib.dump(best_model_hyperopt, 'best_walmart_model_hyperopt.pkl')

['best_walmart_model_hyperopt.pkl']

Загрузка сохраненных моделей из файлов

In [44]:
# Load the models from the files
loaded_model_optuna = joblib.load('best_walmart_model_optuna.pkl')
loaded_model_hyperopt = joblib.load('best_walmart_model_hyperopt.pkl')

Предсказания сохраненных и загруженных обратно моделей на тестовой выборке, вывод на экран полученных при этом метрик

In [45]:
# Make predictions using the loaded models
loaded_model_predictions_optuna = loaded_model_optuna.predict(X_test)
loaded_model_predictions_hyperopt = loaded_model_hyperopt.predict(X_test)

print(f'Loaded Optuna Model R^2 Score: {r2_score(y_test, loaded_model_predictions_optuna)}')
print(f'Loaded Optuna Model RMSE: {root_mean_squared_error(y_test, loaded_model_predictions_optuna)}')

print(f'Loaded Hyperopt Model R^2 Score: {r2_score(y_test, loaded_model_predictions_hyperopt)}')
print(f'Loaded Hyperopt Model RMSE: {root_mean_squared_error(y_test, loaded_model_predictions_hyperopt)}')

Loaded Optuna Model R^2 Score: 0.9997868786591816
Loaded Optuna Model RMSE: 8286.01308021603
Loaded Hyperopt Model R^2 Score: 0.9997638020230121
Loaded Hyperopt Model RMSE: 8723.087552232219


Значения метрик те же, что и до сохранения моделей, таким образом, сохранение и загрузка работают.

Выводы: <br>
На основе датасета Walmart, описывающего пятничные продажи в 45 магазинах соответствующей сети за период в несколько лет, составлена модель регрессии, позволяющая прогнозировать продажи для каждого из этих магазинов в зависимости от следующих параметров: 
- совпадение даты с праздничными днями
- температура воздуха
- стоимость топлива
- текущий индекс потребительских цен
- текущий уровень безработицы <br>
При расчете модели с помощью случайного леса использовался пайплайн.
Для поиска оптимального набора гиперпараметров модели использовались два метода, реализованные в библиотеках optuna и hyperopt.
Проведена кроссвалидация и пробное сохранение моделей в файл и загрузка сохраненных моделей обратно в рабочую среду. После сохранения и загрузки продемонстрирована успешная работа модели на тестовой выборке.

Поиск из 50 попыток в нашем случае занял для optuna около 8'30'', для hyperopt - около 4'40'', т.е. hyperopt на данном датасете быстрее.
При этом разница в достигнутом значениее RMSE не то чтобы весьма значительна - около 8300 ед. для optuna и ~8700 ед. для hyperopt.
При сохранении модели файл у optuna выходит в полтора-три раза больше, чем у hyperopt. При размере в десятки Мб это может иметь значение.

Таким образом, в нашем случае hyperopt показал себя более быстрым и более экономным по отношению к дисковому пространству при сохранении файла модели, при сравнимом качестве поиска оптимальных гиперпараметров.
